In [24]:
import numpy as np


def concatenate(X):
    return np.concatenate([X, np.ones((X.shape[0], 1))], axis=1)


def error(X, y, model):
    return model.predict_proba(X)[:, 1] - y


def prediction(X, model):
    return model.predict_proba(X)[:, 1]


def gradient_(X, y, model, C):
    F = concatenate(X)
    #return F * error(X, y, model)[:, None] + C * w / X.shape[0]
    return F * error(X, y, model)[:, None]


def hessian_(X, model, C):
    probs = prediction(X, model)
    F = concatenate(X)
    H = F.T @ np.diag(probs * (1 - probs)) @ F / F.shape[0] + C * np.eye(
        F.shape[1]) / X.shape[0]
    return H


def inverse_hessian(H):
    return np.linalg.inv(H)

In [25]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]
    
    X_r = X["train"][top_k_index]
    y_r = y["train"][top_k_index]
    X_l = np.delete(X["train"], top_k_index, axis=0)
    y_l = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    #print("prediction", prediction)

    return X_l, y_l, prediction, X_r, y_r



In [26]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction, x_r, y_r = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    return new

In [27]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    for k in range(1, np.sum(y["train"])):
        X_l, y_l, pred_change_if, X_r, y_r = Remove(k, delta_pred, test_idx)
        #_, flip,_ = new_train(k, test_idx, delta_pred)
        
        if old > 0.5 and pred_change_if+old <0.5:
            return k
        elif old < 0.5 and pred_change_if+old > 0.5:
            return k
        
    return None

In [28]:
def recursive_NT(test_idx, old, X_0, y_0, model, l2, I=100, D=1):
    eps = 1 / X["train"].shape[0]
    X_l = X_0
    y_l = y_0
    X_r = X_0
    y_r = y_0
    # Compute IP
    F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)
    new_hessain = hessian_(X_l, model, l2)
    new_inv = inverse_hessian(new_hessain)
    new_grad_train = gradient_(X_r, y_r, model, l2)
    delta_k = -eps * new_inv @ new_grad_train.T
    grad_f = F_dev[test_idx] * (old * (1 - old))
    delta_pred = grad_f @ delta_k

    K_new = y_0.shape[0]
    predicted_change_new = None
    ite = 0
    diff = K_new - 0
    removed_order = []
    while (ite < I and diff > D and K_new != 1):
        ite += 1

        if old > 0.5:
            sorted_index = np.flip(delta_pred.argsort())
        else:
            sorted_index = delta_pred.argsort()

        for k in range(1, y_r.shape[0]):
            top_k_index = sorted_index[:k]
            predicted_change = -np.sum(delta_pred[top_k_index])
            """
            print("K", k)
            print("predicted_change", predicted_change)
            print("old", np.round(old))
            print("K", k)
            print("removed shape", X_r.shape)
            print("left shape", X_l.shape)
            print("delta_pred shape", delta_pred.shape)
            print("predicted_change", predicted_change)
            print("old", np.round(old), "new", np.round(old + predicted_change))
            print()
            """

            if np.round(old) != np.round(old + predicted_change):
                # print("K", k)

                diff = K_new - k
                K_new = k
                predicted_change_new = predicted_change

                index_whole = []
                for idx_r in top_k_index:
                    point = X_r[idx_r]
                    idx_0 = X_dist[str(point.tolist())]
                    index_whole.append(idx_0)

                X_r = X_r[top_k_index]
                y_r = y_r[top_k_index]

                X_l = np.delete(X_0, index_whole, axis=0)
                y_l = np.delete(y_0, index_whole, axis=0)
                # print("removed shape", X_r.shape)
                # print("left shape", X_l.shape)

                # new hessian for the left points
                new_hessain = hessian_(X_l, model, C=l2)
                new_inv = inverse_hessian(new_hessain)
                # new gradient for the removed points
                new_grad_train = gradient_(X_r, y_r, model, C=l2)

                delta_k = -eps * new_inv @ new_grad_train.T
                grad_f = F_dev[test_idx] * (old * (1 - old))
                delta_pred = grad_f @ delta_k
                break

            if k == y_r.shape[0] - 1:
                if K_new == y_0.shape[0] and diff == y_0.shape[0]:
                    return None, predicted_change_new, ite, None, []
                
                return K_new, predicted_change_new, ite, diff, index_whole

    return K_new, predicted_change_new, ite, diff, index_whole

In [29]:
import numpy as np
import warnings
X = {}
y = {}
X["train"] = np.load("X_train.npy")
y["train"] = np.load("y_train.npy")
X["dev"] = np.load("X_test.npy")
y["dev"] = np.load("y_test.npy")
print(X["train"].shape, X["dev"].shape)

(18000, 300) (1000, 300)


In [30]:
from sklearn.linear_model import LogisticRegression
l2 = 1
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.697

In [31]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inv_H = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * inv_H @ gradient_train.T
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k



In [32]:
"""
X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i
    
NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()
    print("old", old)

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change_nt, ite, diff")
    print(K_nt, pred_change_nt, ite, diff)
    print("new", new_nt)
    print("order", len(order))
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_tweet_LR_I100_D1.npy", NT_app_k)
np.save("new_predictions_tweet_LR_I100_D1.npy", new_predictions)
np.save("iterations_tweet_LR_I100_D1.npy", iterations)
np.save("diffs_tweet_LR_I100_D1.npy", diffs)
np.save("order_tweet_LR_I100_D1.npy", order_lists)
"""

'\nX_dist = {}\nfor i in range(X["train"].shape[0]):\n    X_dist[str(X["train"][i].tolist())] = i\n    \nNT_app_k = []\nnew_predictions = []\niterations = []\ndiffs = []\norder_lists = []\n\nfor i in range(X["dev"].shape[0]):\n    test_idx = i\n    print("test_idx", test_idx)\n\n    old = pred[test_idx].item()\n    print("old", old)\n\n    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)\n\n    if pred_change_nt != None:  \n        new_nt = new_train(K_nt, test_idx, delta_pred)\n    else:\n        new_nt = None\n\n        \n    print("K_nt, pred_change_nt, ite, diff")\n    print(K_nt, pred_change_nt, ite, diff)\n    print("new", new_nt)\n    print("order", len(order))\n    print()\n    NT_app_k.append(K_nt)\n    new_predictions.append(new_nt)\n    iterations.append(ite)\n    diffs.append(diff)\n    order_lists.append(order)\n\nnp.save("NT_app_k_tweet_LR_I100_D1.npy", NT_app_k)\nnp.save("new_predictions_tweet_LR_I100_D1.n

# Bert lin

In [33]:

X = {}
y = {}

X["train"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_tweet/train_feature_save.npy")
y["train"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_tweet/train_label_save.npy").squeeze()
X["dev"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_tweet/test_feature_save.npy")
y["dev"] = np.load("/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_tweet/test_label_save.npy").squeeze()
"""
X["train"] = np.load("train_feature_save.npy")
y["train"] = np.load("train_label_save.npy").squeeze()
X["dev"] = np.load("test_feature_save.npy")
y["dev"] = np.load("test_label_save.npy").squeeze()
"""

# In[14]:


print(X["train"].shape, X["dev"].shape, y["train"].shape, y["dev"].shape)

(18000, 768) (1000, 768) (18000,) (1000,)


In [34]:
# In[15]:


l2 = 10

def new_train(k, dev_index, scores):
    X_k, y_k, prediction, x_r, y_r = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=1/l2)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    return new

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]

#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
H = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_H = np.linalg.inv(H)

eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_H @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape


# In[18]:


X_dist = {}
for i in range(X["train"].shape[0]):
    X_dist[str(X["train"][i].tolist())] = i

NT_app_k = []
new_predictions = []
iterations = []
diffs = []
order_lists = []

for i in range(X["dev"].shape[0]):
    test_idx = i
    print("test_idx", test_idx)

    old = pred[test_idx].item()
    print("old", old)

    K_nt, pred_change_nt, ite, diff, order= recursive_NT(test_idx, old, X["train"], y["train"], model, l2, I=100, D=1)

    if pred_change_nt != None:  
        new_nt = new_train(K_nt, test_idx, delta_pred)
    else:
        new_nt = None

        
    print("K_nt, pred_change_nt, ite, diff")
    print(K_nt, pred_change_nt, ite, diff)
    print("new", new_nt)
    print()
    NT_app_k.append(K_nt)
    new_predictions.append(new_nt)
    iterations.append(ite)
    diffs.append(diff)
    order_lists.append(order)

np.save("NT_app_k_tweet_bertT_I100_D1.npy", NT_app_k)
np.save("new_predictions_tweet_bertT_I100_D1.npy", new_predictions)
np.save("iterations_tweet_bertT_I100_D1.npy", iterations)
np.save("diffs_tweet_bertT_I100_D1.npy", diffs)
np.save("order_tweet_bertT_I100_D1.npy", order_lists)


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


test_idx 0
old 0.05797460664469548


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
2523 0.4421060059041533 4 250
new 0.9975406614190568

test_idx 1
old 0.42315594731322376


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
141 -0.1520847394196932 3 11
new 0.5029283207769938

test_idx 998
old 0.6835290526037773


/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K_nt, pred_change_nt, ite, diff
265 -0.18361083795814256 3 31
new 0.4982134610062249

test_idx 999
old 0.732954700504407
K_nt, pred_change_nt, ite, diff
254 -0.23298460128066428 3 34
new 0.5087336617834934



/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [36]:
len(NT_app_k)

1000